In [650]:
import numpy as np
import qiskit
from qiskit.quantum_info import state_fidelity

from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy
import h5py

import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import construct_ising_local_term, reduce_list, X, I2, get_perms
from ansatz_sparse import ansatz_sparse
import rqcopt as oc
from scipy.sparse.linalg import expm_multiply
from qiskit.quantum_info import random_statevector

Lx, Ly = (4, 4)
L = Lx*Ly
t = .1
latt = qib.lattice.TriangularLattice((Lx, Ly), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J, h, g = (1, 0, 2)
hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=100)
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]

X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
Y = np.array([[0, -1j], [1j, 0]])
I2 = np.array([[1, 0], [0, 1]])


perms_1 = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 0, 5, 6, 7, 4, 9, 10, 11, 8, 13, 14, 15, 12]]
perms_2 = [[0, 5, 10, 15, 3, 4, 9, 14, 2, 7, 8, 13, 1, 6, 11, 12], [5, 10, 15, 0, 4, 9, 14, 3, 7, 8, 13, 2, 6, 11, 12, 1]]
perms_3 = [[0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15], [4, 8, 12, 0, 5, 9, 13, 1, 6, 10, 14, 2, 7, 11, 15, 3]]

eigenvalues_sort[0]

-36.15423061323431

In [617]:
"""
    Compressed-Controlled Time Evolution Operator that we optimized previously.
"""
import h5py
import sys
sys.path.append("../../src/brickwall_sparse")
from ansatz_sparse import ansatz_sparse, construct_ccU


perms_qc = [[0, 1], [0, 2]]
Xlists_opts = {}
Vlists = {}
qc_cUs = {}
ts = [0.1, 0.11, 0.12]
for t in ts:
    Vlist = []
    with h5py.File(f'./results/triangularTFIM_ccU_SPARSE_10{g}_Lx4Ly4_t{t}_layers9_niter10_rS1_2hloc.hdf5') as f:
        Vlist =  f["Vlist"][:]
    perms_extended = [[perms_1[0]]] + [perms_1] + [[perms_1[0]], [perms_2[0]]] +\
                        [perms_2] + [[perms_2[0]], [perms_3[0]]] + [perms_3] + [[perms_3[0]]] 
    perms_ext_reduced = [perms_1]  + [perms_2] + [perms_3]
    control_layers = [0, 2, 3, 5, 6, 8]
    
    Xlists_opt = {}
    for i in control_layers:
        with h5py.File(f"./results/triangularTFIM_ccU_SPARSE_10{g}_Lx4Ly4_t{t}_layers25_niter5_rS1_DECOMPOSE_n{len(perms_qc)}_layer{i}.hdf5", "r") as file:
            Xlists_opt[i] = file[f"Xlist_{i}"][:]
        
    Xlists_opts[t] = Xlists_opt
    Vlists[t] = Vlist
    qc_cUs[t] = construct_ccU(L, Vlist, Xlists_opt, perms_extended, perms_qc, control_layers)

qc_cU = qc_cUs[0.11]

In [567]:
from qiskit.quantum_info import random_statevector
from scipy.sparse.linalg import expm_multiply
from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp


for t_ in [.1]:
    qc = qc_cUs[t_]
    state = random_statevector(2**L).data
    qc_ext1 = qiskit.QuantumCircuit(L+1)
    qc_ext1.initialize(state, [i for i in range(L)])
    for i in range(int(t_/t_)):
        qc_ext1.append(qc.to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv1 = execute(transpile(qc_ext1), backend).result().get_statevector().data
    
    qc_ext2 = qiskit.QuantumCircuit(L+1)
    qc_ext2.initialize(state, [i for i in range(L)])
    qc_ext2.x(L)
    for i in range(int(t_/t_)):
        qc_ext2.append(qc.to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv2 = execute(transpile(qc_ext2), backend).result().get_statevector().data

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])
    exact_v1 = np.kron(ket_0, expm_multiply(1j * t_ * hamil, state))
    exact_v2 = np.kron(ket_1, expm_multiply(-1j * t_ * hamil, state))
    #err = (np.linalg.norm(sv1-exact_v1, ord=2) + np.linalg.norm(sv2-exact_v2, ord=2))/2
    err = (1-state_fidelity(sv1, exact_v1) + 1-state_fidelity(sv2, exact_v2))/2

    noise_model = NoiseModel()
    dag = circuit_to_dag(transpile(qc_ext1, basis_gates=noise_model.basis_gates+['unitary', 'initialize', 'cx']))
    count_ops = dag.count_ops()
    
    print(f"t={t_}, Gate Count: ", count_ops['unitary'], " Infid. error: ", err)

t=0.1, Gate Count:  144  Infid. error:  0.001075769661578685


In [618]:
"""

    Exact Initialization + Iterative QPE
    Let's assume we can efficiently encode a DMRG ground state.

"""
import sys
sys.path.append("../../src/qpe")
from qpe import estimate_phases

def norm_mod2pi(theta):
    return np.pi - np.abs((theta%(2*np.pi)) - np.pi) 

def run_QPE(qc_prepared_state, qc_cU, basis_time, init_guess, Ns, final_digit, depolarizing_error, custom_phase=None, 
            hide_print=False, init_digit=-1, T=None):
    theta_prev = -init_guess
    est_prev = init_guess

    x1_error = errors.depolarizing_error(depolarizing_error*0.01, 1)
    x2_error = errors.depolarizing_error(depolarizing_error, 2)
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(x1_error, ['u1', 'u2', 'u3'])
    noise_model.add_all_qubit_quantum_error(x2_error, ['cu3', 'cx', 'str', 'cy', 'cz', 'unitary'])
    
    Es = []
    ests_ = []
    thetas_ = []
    cxss_A = []
    mid_cbits = 0
    mid_errs_A = []
    for j in list(range(init_digit, final_digit+1, 1)):
        if T:
            T = T * 2
        else:
            T = 2**j

        if custom_phase is None:
            theta_prev_ = theta_prev
            counts_real, counts_imag, cxs = estimate_phases(
                                                    L, qc_C, eigenvalues_sort, T/2, basis_time,
                                                    Ns, depolarizing_error, qc_cU, noise_model=noise_model,
                                                    return_counts=True,
                                                    get_cx=True, qasm=False
                                               )[0]
            #cxss_A.append(cxs['unitary'])
            #print('CXs: ', cxss_A[-1])
            print(cxs)
        
            phase_est_real = ((counts_real['0'] if '0' in counts_real else 0) - (counts_real['1'] if '1' in counts_real else 0)) /\
                        ((counts_real['0'] if '0' in counts_real else 0) + (counts_real['1'] if '1' in counts_real else 0))     
            phase_est_imag = ((counts_imag['0'] if '0' in counts_imag else 0) - (counts_imag['1'] if '1' in counts_imag else 0)) /\
                        ((counts_imag['0'] if '0' in counts_imag else 0) + (counts_imag['1'] if '1' in counts_imag else 0))
            phase = phase_est_real + 1j*phase_est_imag
        else:
            T, phase = custom_phase

        if not hide_print:
            print("Exact Phase: ", np.exp(-1j * T * eigenvalues_sort[0]))
            print("Estimated Phase: ", phase)
            print("Estimated Phase Amplitude: ", np.linalg.norm(phase))
                
        if phase.real != 0:
            angle = np.arctan(phase.imag/phase.real) if phase.real>0 else (np.pi + np.arctan(phase.imag/phase.real) if phase.imag>0 else \
                                                                               np.arctan(phase.imag/phase.real) - np.pi)
        else:
            angle = np.pi/2 if phase.imag > 0 else -np.pi/2

    
        if T>1:
            thetas = [(angle + k*2*np.pi)/T for k in range(T+1)]
            norms = np.array([norm_mod2pi(theta - theta_prev) for theta in thetas])
            id_ = np.argmin(norms)
                    
            ests = [-thetas[id_] - coe*np.pi for coe in range(2, 40, 2)]
            est = ests[0]
            for est_ in ests[1:]:
                est = est_ if np.abs(est_-est_prev) < np.abs(est-est_prev) else est
            ests_.append(est)
            thetas_.append(thetas[id_])
            theta_prev = thetas_[-1]
        else:
            thetas = [(angle + k*np.pi)/T for k in range(0, 40, 2)]
            ests = [-theta for theta in thetas]
            est = ests[0]
            for est_ in ests[1:]:
                est = est_ if np.abs(est_-est_prev) < np.abs(est-est_prev) else est
            ests_.append(est)
            thetas_.append(-est)
            theta_prev = thetas_[-1]
    
        est_prev = ests_[-1]
        if not hide_print:
            print("Final Estimation Result, Abs Error: ", np.abs(est_prev-eigenvalues_sort[0]))    
            print("Final Estimation Result, Relative Error: ", np.abs(est_prev-eigenvalues_sort[0])/np.abs(eigenvalues_sort[0]), '\n')    
        mid_errs_A.append(np.abs(est_prev-eigenvalues_sort[0]))
        Es.append(est_prev)
    return Es

In [619]:
"""
    Adiabatic Evolution Implementation.
"""
from qiskit import Aer, transpile, execute
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
X = np.array([[0.,  1.], [1.,  0.]])
Z = np.array([[1.,  0.], [0., -1.]])


def trotter(Lx, Ly, tau, L, J_i, h_i, g_i, J_f, h_f, g_f, lamb, perms):
    L = Lx * Ly
    assert lamb <= 1 and lamb >= 0
    J = lamb*J_f + (1-lamb)*J_i
    g = lamb*g_f + (1-lamb)*g_i
    h = lamb*h_f + (1-lamb)*h_i
    perms1, perms2, perms3 = perms

    qc = qiskit.QuantumCircuit(L)
    hloc1 = J*np.kron(Z, Z)
    hloc2 = g*(np.kron(X, I2)+np.kron(I2, X))/6
    hlocs = (hloc2, hloc1)
        
    perms = perms1+perms2+perms3
    perm_set = perms
    
    indices = [0 , 1,  0]
    coeffs  = [.5, 1, .5]

    Vlist_start = []
    perms = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*tau*hlocs[i]))
        perms.append(perm_set[i])
    Vlist_gates = []
    for V in Vlist_start:
        qc2 = qiskit.QuantumCircuit(2)
        qc2.unitary(V, [0, 1])
        Vlist_gates.append(qc2)
    
    for layer, qc_gate in enumerate(Vlist_gates):
        for perm in perms:
            for j in range(len(perm)//2):
                qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])
    return qc


def construct_ising_local_term_(J, g, ndim, h=0):
    X = np.array([[0.,  1.], [1.,  0.]])
    Z = np.array([[1.,  0.], [0., -1.]])
    I = np.identity(2)
    return J*np.kron(Z, Z) + g*(0.5/ndim)*(np.kron(X, I) + np.kron(I, X)) + h*(0.5/ndim)*(np.kron(Z, I) + np.kron(I, Z))


def run_adiabatic(Lx, Ly, g, T, S, perms, state=None, return_state=False, h_i=0, h_f=0):
    L = Lx*Ly
    tau = 1/S
    t_s = np.linspace(0, T, S*T)
    sch = lambda t, T: np.sin(np.pi*t/(2*T))**2
    
    qc = qiskit.QuantumCircuit(L)
    if state is not None:
        pass
    else:
        qc.x([i for i in range(L)])
        qc.h([i for i in range(L)])
    for s in range(S*T):
        qc.append(trotter(Lx, Ly, tau, L, 0, h_i, g, J, h_f, g, sch(t_s[s], T), perms).to_gate(), [i for i in range(L)])
    
    backend = Aer.get_backend("statevector_simulator")
    qc_ = qiskit.QuantumCircuit(L)
    if state is not None:
        qc_.initialize(state)
    qc_.append(qc.to_gate(), [i for i in range(L)])
    final = execute(transpile(qc_), backend).result().get_statevector().data
    print("AQC: ", [np.linalg.norm(np.vdot(final, eigenvectors_sort[:, i]))**2 for i in range(10)])

    
    if return_state:
        return qc
    else:
        noise_model = NoiseModel()
        dag = circuit_to_dag(transpile(qc_, basis_gates=['unitary', 'u3', 'initialize']))
        count_ops = dag.count_ops()
        print(count_ops)
        return [state_fidelity(final, eigenvectors_sort[:, i]) for i in range(10)], {"gates": count_ops}, final

In [620]:
qc_A = run_adiabatic(Lx, Ly, g, 0, 1, (perms_1, perms_2, perms_3), return_state=True)

AQC:  [0.7649536727294378, 1.710302827813641e-29, 9.750020343460382e-32, 8.093720591287359e-32, 2.1340414202794134e-29, 4.345379437023701e-33, 5.058575369554599e-33, 1.0541939132665848e-32, 7.52316384526264e-33, 1.1943774920738967e-32]


In [634]:
from qiskit import QuantumCircuit

qc_C = qiskit.QuantumCircuit(L+1, 1)
qc_C.append(qc_A.to_gate(), [i for i in range(L)])

T = .1
Es_ccU = run_QPE(qc_C, qc_cUs[T], T, -45, 5000, -1, 0, T=T)

t:  0.1
nsteps:  1
getting counts
{'rz': 36, 'sx': 18, 'unitary': 144, 'measure': 1}
Exact Phase:  (-0.7541685934799764+0.6566808453186632j)
Estimated Phase:  (-0.4172+0.4768j)
Estimated Phase Amplitude:  0.6335566904389851
Final Estimation Result, Abs Error:  0.6777856189255402
Final Estimation Result, Relative Error:  0.015566304960410619 



In [635]:
(31-69)/(31+69) + 1j*(57-47)/(57+47)

(-0.38+0.09615384615384616j)

In [655]:
from qiskit.quantum_info import Statevector

#s = (ground_state * 0.7 + eigenvectors_sort[:, 1] * 0.3)
#s = s/np.linalg.norm(s)
qc_C = qiskit.QuantumCircuit(L+1, 1)
qc_C.x([i for i in range(L)])
qc_C.h([i for i in range(L)])
qc_C.append(qc_A.to_gate(), [i for i in range(L)])


Es_ccU = run_QPE(qc_C, qc_cU, t, -25, 2000, -1, 0, custom_phase=(0.12 * 2,
                                                                 # g = 3 -> 0.19% with fit
                                                                 #0.6991150442477876-0.24166666666666j # t=0.12
                                                                 #-0.44425675675675674-0.4191363251481795j # t=0.1
                                                                 #0.17894736842105263-0.3862433862433862j # t=0.11
                                                                 #0.9737739444430232+0.22751770288018458j # fit for t=0.125

                                                                 # g = 2.5 -> 2.7% with fit
                                                                 #-0.40540540540540543+0.3689839572192513j # t=0.1 v1
                                                                 #-0.32701421800947866+0.19607843137254902j # t=0.1 v2
                                                                 #-0.3010752688172043-0.32608695652173914j # t=0.11
                                                                 #-0.1827956989247312-0.5416666666666666j   # t=0.12
                                                                 #0.19183042251418794-0.9814280865137436j # fit for t=0.125

                                                                 # g = 2 -> 0.14% with fit
                                                                 #0.3958333333333333+0.108j # t=0.1
                                                                 #-0.32972972972972975+0.3125j # t=0.11
                                                                 #-0.30927835051546393+0.3473684210526316j # t=0.12  GOOD
                                                                 #-0.9312687715873161+0.36433291790099226j # fit for t=0.125

                                                                 # g = 1.5 -> 8% with fit
                                                                 #1/9-0.33j # t=0.1
                                                                 #0.22-0.25j # t=0.11
                                                                 #0.3125+0.1276595744680851j # t=0.12
                                                                 #0.9233387839859528+0.3839863148438261j
                                                                ))

Exact Phase:  (-0.7332121255183419+0.6799999845535845j)
Estimated Phase:  (-0.30927835051546393+0.3473684210526316j)
Estimated Phase Amplitude:  0.46509990114185634
Final Estimation Result, Abs Error:  0.39823983814661545
Final Estimation Result, Relative Error:  0.011015027325760299 



In [563]:
"""
    Error bars.
"""

E0 = eigenvalues_sort[0]
t = 0.125
ph = (0.9737739444430232+0.22751770288018458j) * 0.3
std_var = 0.026

phase_real_upper = ph.real + std_var/np.abs(ph)
phase_real_lower = ph.real - std_var/np.abs(ph)

phase_imag_upper = std_var/np.abs(ph) + ph.imag  
phase_imag_lower = -std_var/np.abs(ph) + ph.imag

phases_imag = np.linspace(phase_imag_lower, phase_imag_upper, 100)
phases_real = np.linspace(phase_real_lower, phase_real_upper, 100)

Es = []
for phase_imag in phases_imag:
    for phase_real in phases_real:
        Es.append(run_QPE(qc_C, qc_cU, t, -45, 500, -1, 0, custom_phase=(t * 2, phase_real + 1j * phase_imag), hide_print=True))

print("Stage for t=0.125 succesful? ", np.abs(np.min(Es) - np.max(Es))<4)

# Max ERROR BOUND
np.max(np.array([np.abs(E[0] - E0)/(-E0) for E in Es]))

Stage for t=0.125 succesful?  True


0.03343213503012807

In [564]:
Es_ccU-np.min(Es)

array([1.66612585])

In [565]:
np.max(Es)-Es_ccU

array([1.27558835])

In [625]:
"""
    Now here is to compare the performance of the ccU circuit
    with the 1st and 2nd order Trotter circuits, in terms of 
    gate count vs Trotter error. I demonstrate it on L=8 system.
"""

from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors


def cU_trotter(t, L, dag=False, nsteps=1, trotter_degree=1, trotter_step=0.1):
    c1 = 2*t
    if c1/(2*nsteps) > trotter_step:
        nsteps = int(np.ceil(c1/(2 * trotter_step)))
    t = c1/(2*nsteps)

    perms_v, perms_h = get_perms(Lx, Ly)
    indices = oc.SplittingMethod.suzuki(2, trotter_degree).indices
    coeffs = oc.SplittingMethod.suzuki(2,  trotter_degree).coeffs
    perms_ext = ([perms_v, perms_h])*len(indices)
    
    hloc1 = construct_ising_local_term(J, 0, 0, ndim=2)
    hloc2 = g*(np.kron(X, I2)+np.kron(I2, X))/4
    hlocs = (hloc1, hloc2)

    K = []
    for i, perms in enumerate(perms_ext):
        perm = perms[0]
        K_layer = [None for _ in range(L)]
        for j in range(len(perm)//2):
            K_layer[perm[2*j]] = CYGate
            K_layer[perm[2*j+1]] = CZGate
        K.append(K_layer)
    Vlist_start = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*t*hlocs[i]))
    Vlist_gates = []
    for V in Vlist_start:
        qc2 = qiskit.QuantumCircuit(2)
        qc2.unitary(V, [0, 1], label='str')
        Vlist_gates.append(qc2)


    qc = qiskit.QuantumCircuit(L+1)
    for n in range(nsteps):
        for layer, qc_gate in enumerate(Vlist_gates):

            perms = perms_v
            qc.x(L)
            for j in range(L):
                if K[2*layer][j]:
                    qc.append(K[2*layer][j](), [L, L-1-j])
            qc.x(L)

            for perm in perms:
                for j in range(len(perm)//2):
                    qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])
            qc.x(L)
            for j in range(L):
                if K[2*layer][j]:
                    qc.append(K[2*layer][j](), [L, L-1-j])
            qc.x(L)

            perms = perms_h
            qc.x(L)
            for j in range(L):
                if K[2*layer+1][j]:
                    qc.append(K[2*layer+1][j](), [L, L-1-j])
            qc.x(L)
            
            for perm in perms:
                for j in range(len(perm)//2):
                    qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])

            qc.x(L)
            for j in range(L):
                if K[2*layer+1][j]:
                    qc.append(K[2*layer+1][j](), [L, L-1-j])
            qc.x(L)
    return qc


In [605]:
import rqcopt as oc
import scipy


def f(x, phase_estimates):
    N = len(phase_estimates)
    total_sum = 0
    for i, t in enumerate(phase_estimates[:, 0]):
        Zn = phase_estimates[:, 1][i]
        total_sum += np.abs(Zn - x[0]*np.exp(-1j*t*x[1]))**2
    return total_sum/N

depolarizing_error = 0
x1_error = errors.depolarizing_error(depolarizing_error*0.01, 1)
x2_error = errors.depolarizing_error(depolarizing_error, 2)
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(x1_error, ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(x2_error, ['cu3', 'cx', 'str', 'cy', 'cz', 'unitary'])

Ns = 500
Js = 0
#taus = np.array([0.2])*2
taus = np.array([0.125])*2
lambda_min, lambda_max = (-30, -20)
est = [4, -25]
init_guess = est[1]
cxs = []
mid_cbits = 0

Es = []
ests_ = []
thetas_ = []
cxss_A = []
mid_cbits = 0
mid_errs_A = []

# g = 3
#phase_estimates = [
#    (0.12*2, 0.6991150442477876-0.24166666666666j),
#    (0.1*2, -0.44425675675675674-0.4191363251481795j),
#    (0.11*2, 0.17894736842105263-0.3862433862433862j),
#]

# g = 2.5
#phase_estimates = [
#    (0.12*2, -0.1827956989247312-0.5416666666666666j),
#    (0.11*2, -0.3010752688172043-0.32608695652173914j),
#    (0.1*2, -0.40540540540540543+0.3689839572192513j),
#]

# g = 2
#phase_estimates = [
#    (0.12*2, -0.30927835051546393+0.3473684210526316j),
#    (0.11*2, -0.32972972972972975+0.3125j ),
#    (0.1*2, 0.3958333333333333+0.108j),
#]

# g = 2
phase_estimates = [
    (0.12*2, 0.3125+0.1276595744680851j),
#    (0.11*2, 0.22-0.25j),
    (0.1*2, 1/9-0.33j),
]


phase_estimates = np.array(phase_estimates)
est = scipy.optimize.minimize(lambda x: f(x, phase_estimates), est, bounds=[(est[0]-1, est[0]+1), (lambda_min, lambda_max)]).x

# !!
T = .125
phase_after_fit = np.exp(-1j * T*2 * est[1])
print(f"Phase after fit for t={2*T}: ", phase_after_fit)
print(f"Exact Phase for t={2*T}: ", np.exp(-1j * T*2 * eigenvalues_sort[0]))
phase = phase_after_fit


theta_prev = -init_guess
est_prev = init_guess
if phase.real != 0:
    angle = np.arctan(phase.imag/phase.real) if phase.real>0 else (np.pi + np.arctan(phase.imag/phase.real) if phase.imag>0 else \
                                                                               np.arctan(phase.imag/phase.real) - np.pi)
else:
    angle = np.pi/2 if phase.imag > 0 else -np.pi/2
# You would start from a lower T, then make your way up iteratively till you lose the locked value. T=1 seems to work well! You also need to update your init_guess each iter.

if T>1:
    thetas = [(angle + k*2*np.pi)/T for k in range(T+1)]
    norms = np.array([norm_mod2pi(theta - theta_prev) for theta in thetas])
    id_ = np.argmin(norms)
                    
    ests = [-thetas[id_] - coe*np.pi for coe in range(2, 40, 2)]
    est = ests[0]
    for est_ in ests[1:]:
        est = est_ if np.abs(est_-est_prev) < np.abs(est-est_prev) else est
    ests_.append(est)
    thetas_.append(thetas[id_])
    theta_prev = thetas_[-1]
    
    est_prev = ests_[-1]
    print("Final Estimation Result, Abs Error: ", np.abs(est_prev-eigenvalues_sort[0])) 
    print("Final Estimation Result, Relative Error: ", np.abs(est_prev-eigenvalues_sort[0])/np.abs(eigenvalues_sort[0]), '\n')    
    mid_errs_A.append(np.abs(est_prev-eigenvalues_sort[0]))
    Es.append(est_prev)
    
else:
    thetas = [(angle + k*np.pi)/T for k in range(0, 40, 2)]
    ests = [-theta for theta in thetas]
    est = ests[0]
    for est_ in ests[1:]:
        est = est_ if np.abs(est_-est_prev) < np.abs(est-est_prev) else est
        ests_.append(est)
        thetas_.append(-est)
        theta_prev = thetas_[-1]
    
    est_prev = ests_[-1]
    print("Final Estimation Result, Abs Error: ", np.abs(est_prev-eigenvalues_sort[0])) 
    print("Final Estimation Result, Relative Error: ", np.abs(est_prev-eigenvalues_sort[0])/np.abs(eigenvalues_sort[0]), '\n')    
    mid_errs_A.append(np.abs(est_prev-eigenvalues_sort[0]))
    Es.append(est_prev)


Phase after fit for t=0.25:  (0.9233387839859528+0.3839863148438261j)
Exact Phase for t=0.25:  (0.563683803667364+0.8259906594405851j)
Final Estimation Result, Abs Error:  25.867693104885383
Final Estimation Result, Relative Error:  0.8913571409713315 

